In [ ]:
import ipywidgets as widgets
import hashlib, glob
from IPython.display import display

# hash

How can you check that a copy of a file you send to someone isn't accidentally modified in transit?

[Secure hashes](https://en.wikipedia.org/wiki/Cryptographic_hash_function) are one way to perform such a check.

1. Choose a hashing algorithm.
2. Upload a small file to the dashboard.
3. Generate a hash.
4. [Send](https://wormhole.app/) the file to a friend, and have them repeat steps 1, 2, and 3. (Email works too.)
5. Compare hashes!

In [ ]:
algo = widgets.ToggleButtons(
    options=["md5", "sha1", "sha256"],
    description="Select a hashing algorithm:"
)

uploader = widgets.FileUpload()
select_file = widgets.Select(options="", description ="Select a file to hash:")
files = glob.glob("")
current_file = None
out = widgets.Output()
button = widgets.Button(description = "Generate file hash:")

In [ ]:
# adapted from
# https://stackoverflow.com/a/44873382
# in python 3.11+ you can use hashlib.file_hash() instead 
def shasum(filename, digest):
    h  = hashlib.new(digest)
    b  = bytearray(128*1024)
    mv = memoryview(b)
    with open(filename, 'rb', buffering=0) as f:
        for n in iter(lambda : f.readinto(mv), 0):
            h.update(mv[:n])
    print(f"{digest}: {h.hexdigest()}")
    return f"{digest}: {h.hexdigest()}"

In [ ]:
# adpated from
# https://github.com/fomightez/3Dscatter_plot-binder
@out.capture(clear_output=True, wait=True)
def on_button_clicked(b):
    global uploader, algo, select_file, current_file
    current_file = select_file.value
    filehash = shasum(current_file, algo.value)
    vbox.children = [algo, uploader, select_file, button, out]

def on_file_upload(change):
    global uploader, select_file, current_file
    for details_dict in change["new"]:
        files.append(details_dict["name"])

        with open(details_dict["name"], "wb") as f:
            f.write(details_dict["content"])
    
    current_file = select_file.value
    select_file.options = tuple(files)
    select_file.value

button.on_click(on_button_clicked)
vbox = widgets.VBox([algo, uploader, select_file, button])

In [ ]:
uploader.observe(on_file_upload, "value")
vbox